In [1]:
import sys

sys.path.append('../')

import numpy as np
import gym
import cvxpy as cp
import or_suite
import pickle

In [2]:
algorithm = 'Equal_Allocation'
problem = 'simple'
dir_path = '../data/allocation_%s_%s'%(algorithm,problem)
dir_path = dir_path+'/trajectory.obj'

In [3]:
print(dir_path)

../data/allocation_Equal_Allocation_simple/trajectory.obj


In [4]:
with open(dir_path, 'rb') as f:
    x = pickle.load(f)


In [5]:
x

[{'iter': 0,
  'episode': 1,
  'step': 0,
  'oldState': array([10.,  2.]),
  'action': array([[0.5]]),
  'reward': array([-0.69314718]),
  'newState': array([9., 2.]),
  'info': {'type': array([2])}},
 {'iter': 0,
  'episode': 1,
  'step': 1,
  'oldState': array([9., 2.]),
  'action': array([[0.5]]),
  'reward': array([-0.69314718]),
  'newState': array([8., 2.]),
  'info': {'type': array([2])}},
 {'iter': 0,
  'episode': 1,
  'step': 2,
  'oldState': array([8., 2.]),
  'action': array([[0.5]]),
  'reward': array([-0.69314718]),
  'newState': array([7., 2.]),
  'info': {'type': array([2])}},
 {'iter': 0,
  'episode': 1,
  'step': 3,
  'oldState': array([7., 2.]),
  'action': array([[0.5]]),
  'reward': array([-0.69314718]),
  'newState': array([6., 2.]),
  'info': {'type': array([2])}},
 {'iter': 0,
  'episode': 1,
  'step': 4,
  'oldState': array([6., 2.]),
  'action': array([[0.5]]),
  'reward': array([-0.69314718]),
  'newState': array([5., 2.]),
  'info': {'type': array([2])}},
 {'

In [6]:
def get_envy(X_alg,X_opt,env_config):
    """
    (helper for delta_envy)
    Finds maximum envy of X_alg's allocation by comparing its utility to that of X_opt
    """
    u = env_config['utility_function']
    w = env_config['weight_matrix']
    max_envy=0
    for t,allocation in enumerate(X_alg):
        for theta,row in enumerate(allocation):
            tmp = abs(u(row,w[theta,:])-u(X_opt[t][theta,:],w[theta,:]))
            if tmp >= max_envy:
                max_envy = tmp
    return max_envy

In [7]:
def get_efficiency(X_alg, sizes,env_config):
    """
    (helper for delta_efficiency)
    Finds efficiency by seeing how much of the initial budget was used in X_alg
    """
    B = env_config['init_budget']
    tot_sizes = np.sum(sizes, axis=0)
    num_types,num_commodities = env_config['weight_matrix'].shape
    return sum([B-sum([tot_sizes[theta]*X_alg[t][theta,:] for theta in range(num_types)]) for t in range(len(X_alg))])

In [8]:
def get_proportionality(X_alg,sizes,env_config):
    """
    (helper for delta_proportionality)
    Finds proportionality by calculating envy w.r.t a completely equal allocation
    """
    B = env_config['init_budget']
    tot_size = np.sum(sizes)
    u = env_config['utility_function']
    w = env_config['weight_matrix']
    max_prop=0
    for t,allocation in enumerate(X_alg):
        for theta,row in enumerate(allocation):
            tmp = abs(u(row,w[theta,:])-u(B/tot_size,w[theta,:]))
            if tmp >= max_prop:
                max_prop = tmp
    return max_prop

In [9]:
#SEANS CODE FOR GENERATING SOLVER FOR OFFLINE PROBLEM
def generate_cvxpy_solve(num_types, num_resources):
    """
    Creates a generic solver to solve the offline resource allocation problem
    
    Inputs: 
        num_types - number of types
        num_resources - number of resources
    Returns:
        prob - CVXPY problem object
        solver - function that solves the problem given data
    """
    x = cp.Variable(shape=(num_types,num_resources))
    sizes = cp.Parameter(num_types, nonneg=True)
    weights = cp.Parameter((num_types, num_resources), nonneg=True)
    budget = cp.Parameter(num_resources, nonneg=True)
    objective = cp.Maximize(cp.log(cp.sum(cp.multiply(x, weights), axis=1)) @ sizes)
    constraints = []
    constraints += [0 <= x]
    for i in range(num_resources):
        constraints += [x[:, i] @ sizes <= budget[i]]
    # constraints += [x @ sizes <= budget]
    prob = cp.Problem(objective, constraints)
    def solver(true_sizes, true_weights, true_budget):
        sizes.value = true_sizes
        weights.value = true_weights
        budget.value = true_budget
        prob.solve()
        return prob.value, np.around(x.value, 5)
    return prob, solver

In [10]:
def offline_opt(budget, size, weights, solver):
    """
    Uses solver from generate_cvxpy_solve and applies it to values
    
    Inputs:
        budget: initial budget for K commodities
        size: 2D numpy array of sizes of each type at each location
        weights: 2D numpy array containing the demands of each type
    """
    tot_size = np.sum(size, axis=0)
    _, x = solver(tot_size, weights, budget)
    allocation = np.zeros((size.shape[0], weights.shape[0], weights.shape[1]))
    for i in range(size.shape[0]):
        allocation[i,:,:] = x
    return allocation

In [17]:
def delta_envy(traj, env_config):
    """
    Calculates the delta_envy metric given the trajectory of a given algorithm
    
    Inputs:
        traj: trajectory of an algorithm, stored as a list of dictionaries
        env_config: configuration of the environment
    Returns:
        final_avg_envies: array of the average envy for each episode
        
    WHY ARE EPISODES INDEXED BY 1 PLEASE FIX
    """
    import cvxpy as cp
    num_iter = traj[-1]['iter']+1
    num_eps = traj[-1]['episode']+1
    num_steps = traj[-1]['step']+1
    #print('Iters: %s, Eps: %s, Steps: %s'%(num_iter,num_eps,num_steps))
    num_types,num_commodities = traj[-1]['action'].shape 
    final_avg_envies = np.zeros(num_eps-1)
    
    for iteration in range(num_iter):      
        iteration_traj = list(filter(lambda d: d['iter']==iteration, traj))
        
        for ep in range(1,num_eps):
            ep_traj = list(filter(lambda d: d['episode']==ep, traj))
            sizes = np.zeros((num_steps,num_types))

            for idx,step_dict in enumerate(ep_traj):
                size = step_dict['info']['type']
                sizes[idx,:] = size
                   
            prob, solver = generate_cvxpy_solve(num_types,num_commodities)
            X_opt = offline_opt(env_config['init_budget'],sizes,env_config['weight_matrix'],solver)
            X_alg = np.zeros((num_steps,num_types,num_commodities))
            
            for idx,step_dict in enumerate(ep_traj):
                X_alg[idx,:,:] = step_dict['action']
            
            envy = get_envy(X_alg,X_opt,env_config)
            final_avg_envies[ep-1] += (1/num_iter)*envy
            #print("Envy for episode %s: %s"%(ep,envy))
            
    return np.mean(final_avg_envies)

In [18]:
def delta_efficiency(traj, env_config):
    """
    Calculate the efficiency (waste) of an algorithm given its trajectory
    
    Inputs:
        traj: trajectory of an algorithm, stored as a list of dictionaries
        env_config: configuration of the environment
    Returns:
        final_avg_efficiency: array containing average waste per episode 

    WHY ARE EPISODES 1 INDEXED BUT ITERATIONS AND STEPS NOT PLEASE FIX
    """
    import cvxpy as cp
    num_iter = traj[-1]['iter']+1
    num_eps = traj[-1]['episode']+1
    num_steps = traj[-1]['step']+1
    #print('Iters: %s, Eps: %s, Steps: %s'%(num_iter,num_eps,num_steps))
    num_types,num_commodities = traj[-1]['action'].shape 
    final_avg_efficiency = np.zeros(num_eps-1)
    for iteration in range(num_iter):      
        iteration_traj = list(filter(lambda d: d['iter']==iteration, traj))

        for ep in range(1,num_eps):
            ep_traj = list(filter(lambda d: d['episode']==ep, traj))
            sizes = np.zeros((num_steps,num_types))

            for idx,step_dict in enumerate(ep_traj):
                size = step_dict['info']['type']
                sizes[idx,:] = size
            
            X_alg = np.zeros((num_steps,num_types,num_commodities))
            
            for idx,step_dict in enumerate(ep_traj):
                X_alg[idx,:,:] = step_dict['action']
            
            eff = get_efficiency(X_alg,sizes,env_config)
            final_avg_efficiency[ep-1] += (1/num_iter)*eff
            #print("Efficiency for episode %s: %s"%(ep,eff))

    return np.mean(final_avg_efficiency)

In [19]:
def delta_proportionality(traj, env_config):
    """
    Calculate the proportionality (distance to equal allocation) at each episode
    
    Inputs:
        traj: trajectory of an algorithm, stored as a list of dictionaries
        env_config: configuration of the environment
    Returns:
        final_avg_efficiency: array containing average waste per episode 
    
    WHY ARE EPISODES 1 INDEXED BUT ITERATIONS AND STEPS NOT PLEASE FIX
    """
    import cvxpy as cp
    
    num_iter = traj[-1]['iter']+1
    num_eps = traj[-1]['episode']+1
    num_steps = traj[-1]['step']+1
    #print('Iters: %s, Eps: %s, Steps: %s'%(num_iter,num_eps,num_steps))
    num_types,num_commodities = traj[-1]['action'].shape 
    final_avg_efficiency = np.zeros(num_eps-1)
    
    for iteration in range(num_iter):      
        iteration_traj = list(filter(lambda d: d['iter']==iteration, traj))

        for ep in range(1,num_eps):

            ep_traj = list(filter(lambda d: d['episode']==ep, traj))
            sizes = np.zeros((num_steps,num_types))

            for idx,step_dict in enumerate(ep_traj):
                size = step_dict['info']['type']
                sizes[idx,:] = size

            X_alg = np.zeros((num_steps,num_types,num_commodities))
            
            for idx,step_dict in enumerate(ep_traj):
                X_alg[idx,:,:] = step_dict['action']
            
            prop = get_proportionality(X_alg,sizes,env_config)
            final_avg_efficiency[ep-1] += (1/num_iter)*prop
            #print("Proportionality for episode %s: %s"%(ep,prop))
            
    return np.mean(final_avg_efficiency)

In [20]:
def delta_OPT(traj, env_config):
    """
    Calculates the distance to X_opt w.r.t supremum norm
    
    Inputs:
        traj: trajectory of an algorithm, stored as a list of dictionaries
        env_config: configuration of the environment
    Returns:
        final_avg_dist: array of the average dist to X_opt for each episode
        
    WHY ARE EPISODES INDEXED BY 1 PLEASE FIX
    """
    import cvxpy as cp
    num_iter = traj[-1]['iter']+1
    num_eps = traj[-1]['episode']+1
    num_steps = traj[-1]['step']+1
    #print('Iters: %s, Eps: %s, Steps: %s'%(num_iter,num_eps,num_steps))
    num_types,num_commodities = traj[-1]['action'].shape 
    final_avg_dist = np.zeros(num_eps-1)
    
    for iteration in range(num_iter):      
        iteration_traj = list(filter(lambda d: d['iter']==iteration, traj))
        
        for ep in range(1,num_eps):
            ep_traj = list(filter(lambda d: d['episode']==ep, traj))
            sizes = np.zeros((num_steps,num_types))

            for idx,step_dict in enumerate(ep_traj):
                size = step_dict['info']['type']
                sizes[idx,:] = size
                   
            prob, solver = generate_cvxpy_solve(num_types,num_commodities)
            X_opt = offline_opt(env_config['init_budget'],sizes,env_config['weight_matrix'],solver)
            X_alg = np.zeros((num_steps,num_types,num_commodities))
            
            for idx,step_dict in enumerate(ep_traj):
                X_alg[idx,:,:] = step_dict['action']
            
            dist = np.max(np.absolute(X_opt-X_alg))
            final_avg_dist[ep-1] += (1/num_iter)*dist
            #print("Dist to OPT for episode %s: %s"%(ep,dist))
            
    return np.mean(final_avg_dist)

In [21]:
ENV_CONFIG = {'K':1,
  'num_rounds':10,
  'weight_matrix': np.array([[1]]),
  'init_budget': np.array([10.]),
  'utility_function': lambda x,theta: x,
  'type_dist': lambda i : np.array([2])
}
envy = delta_envy(x,env_config=ENV_CONFIG)
efficiency = delta_efficiency(x, env_config=ENV_CONFIG)
proportionality = delta_proportionality(x, env_config=ENV_CONFIG)
dist_to_OPT = delta_OPT(x,env_config=ENV_CONFIG)

C:\ProgramData\Anaconda3\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:163: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
C:\ProgramData\Anaconda3\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:163: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
C:\ProgramData\Anaconda3\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:163

In [22]:
print("Envy: %s"%envy)
print("Efficiency: %s"%efficiency)
print("Proportionality: %s"%proportionality)
print("Distance to OPT: %s"%dist_to_OPT)

Envy: 0.0
Efficiency: 0.0
Proportionality: 0.0
Distance to OPT: 0.0
